In [ ]:
#Setup
import boto3
import sagemaker
region = boto3.Session().region_name
client = boto3.client("sagemaker", region_name=region)

#Role to give SageMaker permission to access AWS services.
sagemaker_role = sagemaker.get_execution_role()

#Get model from S3
model_url = "s3://bucketgrad2/path/to/ecg_classifier.tar.gz"

#Get container image (prebuilt example)
from sagemaker import image_uris
container = image_uris.retrieve("xgboost", region, "0.90-1")

#Create model
model_name = "ecgclassifier"

response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    Containers = [{
        "Image": container,
        "Mode": "SingleModel",
        "ModelDataUrl": model_url,
    }]
)

In [ ]:
response = client.create_endpoint_config(
   EndpointConfigName="ecgclassifier",
   #KmsKeyId="arn:aws:kms:us-east-1:123456789012:key/143ef68f-76fd-45e3-abba-ed28fc8d3d5e",#issue maker
   ProductionVariants=[
        {
            "ModelName": "ecgclassifier",
            "VariantName": "AllTraffic",
            "ServerlessConfig": {
                "MemorySizeInMB": 2048,
                "MaxConcurrency": 9,
                "ProvisionedConcurrency": 5,
            }
        } 
    ]
)

In [ ]:
response = client.create_endpoint(
    EndpointName="ecgclassifier",
    EndpointConfigName="ecgclassifier"
)